In [1]:
import numpy as np

import chromadb
from chromadb.utils import embedding_functions

from chromadb.config import Settings

chroma_client = chromadb.HttpClient(host='localhost', port = 8083, settings=Settings(allow_reset=True, anonymized_telemetry=False))

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")

collection = chroma_client.get_or_create_collection(name="transcripts_mililm_l6_v2", embedding_function=sentence_transformer_ef)


/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
results = collection.query(
    query_texts=["Tesla transcript"],
    n_results=1,
    where={"symbol": 'TSLA'},
    include=['metadatas', 'documents', 'embeddings']
)

In [3]:
results

{'ids': [['2022Q4TSLA']],
 'distances': None,
 'embeddings': [[[-0.11051055788993835,
    0.06811205297708511,
    0.06706127524375916,
    -0.019170356914401054,
    -0.004925669636577368,
    0.025928430259227753,
    -0.014016933739185333,
    0.04714864864945412,
    0.0854259729385376,
    -0.02157697081565857,
    -0.04010038077831268,
    0.01404737401753664,
    0.036157649010419846,
    -0.06835713982582092,
    -0.016202906146645546,
    0.008394897915422916,
    -0.032223060727119446,
    -0.10664141178131104,
    -0.08782286942005157,
    0.06270056962966919,
    0.029815489426255226,
    -0.012814322486519814,
    -0.011409646831452847,
    0.025328218936920166,
    0.014103294350206852,
    -0.03687543049454689,
    -0.06192099303007126,
    0.03471729904413223,
    -0.04742094874382019,
    0.013960786163806915,
    -0.03184268996119499,
    0.03233993425965309,
    0.0010322188027203083,
    0.024220217019319534,
    -0.020432548597455025,
    0.05170146003365517,
    0

In [4]:
results['documents']

[["Martin Viecha: Good afternoon everyone and welcome to Tesla's Fourth Quarter 2022 Q&A Webcast. My name is Martin Viecha, VP of Investor Relations and I'm joined today by Elon Musk, Zachary Kirkhorn and a number of other executives. Our Q4 results were announced at about 3:00 P.M. Central Time in the update deck we published at the same link as this webcast. During this call, we will discuss our business outlook and make forward-looking statements. These comments are based on our predictions and expectations as of today. Actual events or results could differ materially due to a number of risks and uncertainties, including those mentioned in our most recent filings with the SEC. During the Q&A session portion of today’s call, please limit yourself to one question and one follow-up. Please use the raise hand button to join the question queue.  But before we jump into Q&A, Elon has some opening remarks. Elon?\nElon Musk: Thank you, Martin. So 2022 -- just going through the 2022 recap. I

In [5]:
ls = []

for d, m in zip(results['documents'][0], results['metadatas'][0]):
    dc = {
        'text' : d
    }
    dc.update(**m)
    
    ls.append(dc)
    
    dc = {}

In [6]:
ls

[{'text': "Martin Viecha: Good afternoon everyone and welcome to Tesla's Fourth Quarter 2022 Q&A Webcast. My name is Martin Viecha, VP of Investor Relations and I'm joined today by Elon Musk, Zachary Kirkhorn and a number of other executives. Our Q4 results were announced at about 3:00 P.M. Central Time in the update deck we published at the same link as this webcast. During this call, we will discuss our business outlook and make forward-looking statements. These comments are based on our predictions and expectations as of today. Actual events or results could differ materially due to a number of risks and uncertainties, including those mentioned in our most recent filings with the SEC. During the Q&A session portion of today’s call, please limit yourself to one question and one follow-up. Please use the raise hand button to join the question queue.  But before we jump into Q&A, Elon has some opening remarks. Elon?\nElon Musk: Thank you, Martin. So 2022 -- just going through the 2022 

In [7]:
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.schema.document import Document

text_splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=100)

texts = text_splitter.create_documents(results['documents'][0], results['metadatas'][0])
split_texts = text_splitter.split_documents(texts)

In [9]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.llm import LLMChain

from langchain import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import GPT4All
from langchain.prompts import PromptTemplate

In [10]:
local_path = (
    "../llm_models/gpt4all-falcon-q4_0.gguf"  # replace with your desired local file path
)

# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=False)

In [ ]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(temperature=0)


[autoreload of langchain_core failed: Traceback (most recent call last):
  File "/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/Users/michieldekoninck/.pyenv/versions/3.10.6/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/michieldekoninck/.pyenv/versions/3.10.6/envs/finance/lib/python3.10/site-packages/langchain_core/__init__.py", line 3, in <module>
    from langchain_core._api import (
I

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

we'll first map each document to an individual summary using an LLMChain. Then we'll use a ReduceDocumentsChain to combine those summaries into a single global summary

In [11]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain

# Map
map_template = """The following is a set of transcripts
{docs}
Based on this list of transcripts, create a summary of the most import themes.
"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)


In [22]:
# Reduce
reduce_template = """The following is themes from a stock transcript:
{docs}
Take these and distill it into a final, consolidated summary of the main themes. 
Overall successes:
-
-
..
Overall challenges:
-
-
.."""
reduce_prompt = PromptTemplate.from_template(reduce_template)

In [23]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain


# Run chain
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

# Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Combines and iteratively reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    # This is final chain that is called.
    combine_documents_chain=combine_documents_chain,
    # If documents exceed context for `StuffDocumentsChain`
    collapse_documents_chain=combine_documents_chain,
    # The maximum number of tokens to group documents into.
    token_max=2000,
)

In [24]:
# Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    # Map chain
    llm_chain=map_chain,
    # Reduce chain
    reduce_documents_chain=reduce_documents_chain,
    # The variable name in the llm_chain to put the documents in
    document_variable_name="docs",
    # Return the results of the map steps in the output
    return_intermediate_steps=False,
)

In [26]:
split_texts[10:20]

[Document(page_content="Elon Musk: Yes. Something that I think some of these smart retail investors understand, but I think a lot of others maybe don't is that the -- every time we sell a car, it has the ability, just from uploading software to have full self-driving enabled and full self-driving is obviously getting better very rapidly. So that's actually a tremendous upside potential because all of those cars, with a few exceptions -- I mean, only a small percentage of cars don't have Hardware 3. So that means that there's millions of cars were full self-driving can be sold at essentially 100% gross margin. And the value of it grows as the autonomous capability grows. And then when it becomes fully autonomous, that is a value increase in the fleet. That might be the biggest asset value increase of anything in history. Yeah.\nMartin Viecha: Thank you. Let's go to the next investor question. Since Elon started political influencing, polls from Morning Consult and YouGov show…\nElon Mus

In [25]:
print(map_reduce_chain.run(split_texts[10:20]))


Overall opportunities:
-
-
..
Overall threats:
-
-
..
Overall takeaways:
-
-
..
Overall lessons learned:
-
-
..
Overall key takeaways:
-
-
..
Overall key learnings:
-
-
..
Overall key insights:
-
-
..
Overall key observations:
-
-
..
Overall key findings:
-
-
..
Overall key recommendations:
-
-
..
Overall key conclusions:
-
-
..
Overall key implications:
-
-
..
Overall key issues:
-
-
..
Overall key problems:
-
-
..
Overall key risks:
-
-
..
Overall key threats:
-
-
..
Overall key opportunities:
-
-
..
Overall key challenges:
-
-
..
Overall key takeaways:
-
-
..
Overall key lessons learned:
-
-
..
Overall key insights:
-
-
..
Overall key observations:
-
-
..
Overall key recommendations:
-
-
..
Overall key conclusions
Overall opportunities:
-
-
..
Overall threats:
-
-
..
Overall takeaways:
-
-
..
Overall lessons learned:
-
-
..
Overall key takeaways:
-
-
..
Overall key learnings:
-
-
..
Overall key insights:
-
-
..
Overall key observations:
-
-
..
Overall key findings:
-
-
..
Overall k